In [21]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D, BatchNormalization, LeakyReLU,Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [22]:
dataset_path = "Data\\SOCOFing\\Real"
img_height, img_width = 96, 96

data_paths = [
    "Data\\SOCOFing\\Real",
    "Data\\SOCOFing\\Altered\\Altered-Easy"
]
def preprocess_images(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (img_width, img_height))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=-1)  # Add channel dimension
    return img


def load_dataset(data_paths):
    images = []
    labels = []
    for path in data_paths:
        for file in os.listdir(path):
            if file.endswith(".BMP"):
                img_path = os.path.join(path, file)
                img = preprocess_images(img_path)
                images.append(img)
                label = int(file.split("__")[0]) - 1
                labels.append(label)
    return np.array(images), np.array(labels)

images, labels = load_dataset(data_paths)


label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

In [26]:

def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), input_shape=(img_height, img_width, 1)),
        MaxPooling2D((2, 2)),
        
        Conv2D(64, (3, 3)),
        MaxPooling2D((2, 2)),
        
        Conv2D(128, (3, 3)),
        MaxPooling2D((2, 2)),
        
        Conv2D(256, (3, 3)),
        MaxPooling2D((2, 2)),
        
        GlobalAveragePooling2D(),
        
        Dense(512, activation='relu'),
        Dropout(0.5),
        
        Dense(256, activation='relu'),
        Dropout(0.5),
        
        Dense(600, activation='softmax')  # Lớp đầu ra cho phân loại
    ])
    return model
model = create_model()

In [27]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
checkpoint = ModelCheckpoint('model.keras', save_best_only=True, monitor='val_loss', mode="min")
early_stopping = EarlyStopping(monitor = 'val_accuracy', patience = 10, mode="max",restore_best_weights=True)

In [28]:
model.fit(images, labels_categorical, epochs=100, batch_size=32, validation_split=0.2, callbacks=[checkpoint, early_stopping])
model.save_weights('fingerprint_model_weights.weights.h5')

Epoch 1/100
599/599 ━━━━━━━━━━━━━━━━━━━━ 65s 104ms/step - accuracy: 0.0015 - loss: 6.3701 - val_accuracy: 0.0000e+00 - val_loss: 8.1551
Epoch 2/100
599/599 ━━━━━━━━━━━━━━━━━━━━ 62s 103ms/step - accuracy: 0.0062 - loss: 6.1062 - val_accuracy: 0.0000e+00 - val_loss: 7.5958
Epoch 3/100
599/599 ━━━━━━━━━━━━━━━━━━━━ 70s 117ms/step - accuracy: 0.0135 - loss: 5.6982 - val_accuracy: 0.0000e+00 - val_loss: 7.4010
Epoch 4/100
599/599 ━━━━━━━━━━━━━━━━━━━━ 64s 107ms/step - accuracy: 0.0214 - loss: 5.3298 - val_accuracy: 0.0000e+00 - val_loss: 6.7146
Epoch 5/100
599/599 ━━━━━━━━━━━━━━━━━━━━ 61s 101ms/step - accuracy: 0.0377 - loss: 5.0004 - val_accuracy: 2.0890e-04 - val_loss: 6.4417
Epoch 6/100
599/599 ━━━━━━━━━━━━━━━━━━━━ 61s 101ms/step - accuracy: 0.0569 - loss: 4.6694 - val_accuracy: 2.0890e-04 - val_loss: 5.9456
Epoch 7/100
599/599 ━━━━━━━━━━━━━━━━━━━━ 61s 101ms/step - accuracy: 0.0840 - loss: 4.3353 - val_accuracy: 2.0890e-04 - val_loss: 5.7948
Epoch 8/100
599/599 ━━━━━━━━━━━━━━━━━━━━ 61s 102

In [30]:
model.load_weights('fingerprint_model_weights.weights.h5')
# Hàm trích xuất và lưu trữ đặc trưng
def extract_and_save_features(model, dataset_path):
    images = []
    labels = []
    for file in os.listdir(dataset_path):
        if file.endswith(".BMP"):
            img_path = os.path.join(dataset_path, file)
            img = preprocess_images(img_path)
            images.append(img)
            label = int(file.split("__")[0]) - 1
            labels.append(label)
    
    images = np.array(images)
    features = model.predict(images)
    
    np.save('fingerprint_features.npy', features)
    np.save('fingerprint_labels.npy', labels)

dataset_path = "Data\\SOCOFing\\Real"
extract_and_save_features(model, dataset_path)

188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step


In [38]:
from sklearn.metrics.pairwise import cosine_similarity
# Hàm thêm dấu vân tay mới
# def add_new_fingerprint(model, new_fingerprint_path, new_label):
#     # Tiền xử lý ảnh mới
#     new_image = preprocess_images(new_fingerprint_path)
#     new_image = np.expand_dims(new_image, axis=0)
    
#     # Trích xuất đặc trưng của ảnh mới
#     new_feature = model.predict(new_image)
    
#     # Tải đặc trưng và nhãn hiện tại
#     features = np.load('fingerprint_features.npy')
#     labels = np.load('fingerprint_labels.npy')
    
#     # Thêm đặc trưng và nhãn mới vào cơ sở dữ liệu
#     features = np.vstack([features, new_feature])
#     labels = np.append(labels, new_label)
    
#     # Lưu lại cơ sở dữ liệu cập nhật
#     np.save('fingerprint_features.npy', features)
#     np.save('fingerprint_labels.npy', labels)

# new_fingerprint_path = "path/to/new/fingerprint.BMP"
# new_label = 601  # Giả sử đây là nhãn của dấu vân tay mới
# add_new_fingerprint(model, new_fingerprint_path, new_label)

# Hàm xác thực dấu vân tay
def verify_fingerprint(fingerprint_image_path):
    # Tiền xử lý ảnh đầu vào
    input_image = preprocess_images(fingerprint_image_path)
    input_image = np.expand_dims(input_image, axis=0)

    # Trích xuất đặc trưng
    input_feature = model.predict(input_image).flatten()

    # Tải đặc trưng và nhãn hiện tại
    features = np.load('fingerprint_features.npy')
    labels = np.load('fingerprint_labels.npy')

    # Tính toán độ tương đồng cosine
    similarities = cosine_similarity([input_feature], features)

    # Tìm đặc trưng có độ tương đồng cao nhất
    best_match_index = np.argmax(similarities)
    best_match_label = labels[best_match_index]

    print("Best match label:", best_match_label)
    print("Best match similarity:", similarities[0][best_match_index])

    # Kiểm tra ngưỡng để xác định khớp hay không
    if similarities[0][best_match_index] > 0.85:
        print(f"Unlock successful! Matched label: {best_match_label}")
    else:
        print("Unlock failed! No matching fingerprint found.")

# Ví dụ sử dụng hàm xác thực
verify_fingerprint("Data\\SOCOFing\\Altered\\Altered-Medium\\3__M_Left_index_finger_CR.BMP")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Best match label: 2
Best match similarity: 0.97214985
Unlock successful! Matched label: 2
